In [ ]:
# 코랩 사용 시 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ROOT_PATH 확인 
import os

if os.getcwd() == '/content' :
  # 구글 드라이브 사용 시 
  ROOT_PATH = "/content/drive/MyDrive/AIVLE_3rd[Team]"
else :
  ROOT_PATH = os.path.abspath('..')

# 비디오 파일이 저장되어 있는 폴더
VIDEO_PATH = ROOT_PATH + "/video"

# 모델 학습을 위한 데이터 
TRAIN_PATH = ROOT_PATH + "/crop"
# 모델 예측을 위한 test 데이터
TEST_PATH = ROOT_PATH + "/test"
 
# 모델 파일
MODEL_PATH = ROOT_PATH + '/model'

In [ ]:
# 압축해제 
if not os.path.exists(TEST_PATH + '/crop') : 
  os.mkdir(TEST_PATH + '/crop')

zip_file = ROOT_PATH + '/test_crop.zip'

with zipfile.ZipFile(zip_file) as z:
    z.extractall(TEST_PATH + '/crop')
    
os.remove(zip_file)

In [ ]:
# 필요 라이브러리 불러오기.
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# 테스트 해보기
batch_size = 16
img_height = 26
img_width = 196

test_datagen = ImageDataGenerator(
  rescale = 1./ 255,
)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    batch_size=batch_size,
    target_size=(img_height, img_width),
    class_mode='categorical',
    shuffle=False
)

In [ ]:
def my_model_predict(test_generator, MODEL_PATH):

  import pandas as pd
  import tensorflow as tf
  from tensorflow import keras
  import re

  # 학습된 모델 불러오기
  loaded_model = tf.keras.models.load_model(MODEL_PATH + '/[팀] 미니프로젝트3차_1반_03조_crop.h5')

  # 불러온 모델로 예측하기
  pred = loaded_model.predict(test_generator)
  y_pred = pred.argmax(axis=1)

  # 예측 결과를 저장한 데이터 프레임 생성하기
  second = []
  filename = test_generator.filenames
  for f in filename:
    second.append(int(re.sub(r'[^0-9]', '', f)))
  predict_df = pd.DataFrame({'time' : second, 'class' : y_pred})
  predict_df['class'] = predict_df['class'].map({0 : 'boss',
                                                  1 : 'chrous',
                                                  2 : 'discussion',
                                                  3 : 'enter',
                                                  4 : 'opening_ending'})
  predict_df = predict_df.sort_values('time', ignore_index=True)
  
  return predict_df

In [ ]:
predict_df = my_model_predict(test_generator, MODEL_PATH)

In [ ]:
predict_df

In [ ]:
predict_df.to_csv(ROOT_PATH + '/전처리1_crop_predict_result.csv')